# Bibliotecas e Configurações

1. Requer:
    * `./data/GlobalLandTemperaturesByCity.csv`;
    * `./data/co2_data.csv`.

<br />

2. É necessário possuir o **Java SE Development Kit 11** (JDK) instalado;
    * Outras versões apresentam problemas;
    * Possivel baixar o `jdk-11.0.16` através desse *[link](https://gist.github.com/wavezhang/ba8425f24a968ec9b2a8619d7c2d86a6)*.

<br />

3. Além disso, é necessário definir a variável de ambiente `JAVA_HOME` cujo valor é o diretório de instalação do JDK;
    * Exemplo: `C:\Progra~1\Java\jdk-11.0.16`;
    * Em que `Progra~1` é, na verdade, a pasta `Program Files`.

<br />

4. Definir também as variáveis de ambiente:
    * `PYSPARK_DRIVER_PYTHON` = `jupyter`;
    * `PYSPARK_PYTHON` = `python`.

<br />

5. Além disso tudo, é necessário o Spark + Hadoop para salvar arquivos;
    1. Assim sendo, efetue o download da versão 3.2.2 do [Spark](https://spark.apache.org/downloads.html);
    2. Extraia os arquivos para um diretório desejado do computador (local de instalação);
    3. Dentro desse diretório de instalação do Spark crie o diretório do Haddop: `<spark_path>\hadoop\bin`;
    4. Efetuar o download da versão 3.3.1 do [Hadoop](https://github.com/kontext-tech/winutils)
        * Aqui existe uma gambiarra:
        1. Baixar o repositório como ZIP;
        2. Extrair todos os arquivos da respectiva versão na pasta `bin` recém criada;
        3. Dentro dessa pasta `bin`, criar uma nova pasta `bin` e repetir o passo anterior.

<br />

6. Definir as variáveis de ambiente:
    * `SPARK_HOME`: diretório de instalação do Spark;
    * `HADOOP_HOME` = `%SPARK_HOME%\hadoop\bin`.

<br />

7. Adicionar os caminhos dessa variáveis (`SPARK_HOME` E `HADOOP_HOME`) ao Path;

<br />

8. Será necessário reiniciar o computador após definir as variáveis de ambiente e alterar o Path.
    * Há abaixo uma verificação para atestar se é possível ler todas as variáveis necessárias.

In [5]:
import os

environ_vars = [
    'JAVA_HOME',
    'PYSPARK_DRIVER_PYTHON',
    'PYSPARK_PYTHON',
    'SPARK_HOME',
    'HADOOP_HOME'
]

for var in environ_vars:
    if not os.environ.get(var):
        print(f'AVISO: Variável {var} não encontrada.')

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.utils import AnalysisException

# Leitura

In [7]:
spark = SparkSession.builder.getOrCreate()

tpr_data = spark.read.options(header='True').csv('./data/GlobalLandTemperaturesByCity.csv')
co2_data = spark.read.options(header='True').csv('./data/co2_data.csv')

In [8]:
tpr_data.show(5)

print('Número de Registros:', tpr_data.count())

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows

None
Registros: 8599212


*Dataset* de temperaturas:

* `dt` (string): data da amostra;
* `AverageTemperature` (float): temperatura média medida;
* `AverageTemperatureUncertainty` (float): incerteza associada a temperatura medida;
* `City` (string): cidade de origem da amostra;
* `Country` (string): país de origem da amostra;
* `Latitude` (string): graus de latitude norte da amostra;
* `Longitude` (string):  graus de longitude leste da amostra;

In [9]:
co2_data.show(5)

print('Núemro de registros:', co2_data.count())

+----------+------+------+-----------------+
|     Times|LatDim|LonDim|            value|
+----------+------+------+-----------------+
|1850-01-01|     0|     0|288.1340637207031|
|1850-01-01|     0|     1|288.1340637207031|
|1850-01-01|     0|     2|288.1340637207031|
|1850-01-01|     0|     3|288.1340637207031|
|1850-01-01|     0|     4|288.1340637207031|
+----------+------+------+-----------------+
only showing top 5 rows

None
Registros: 127526400


*Dataset* de temperaturas:

* `Times` (string): data da amostra;
* `LatDim` (int): graus de latitude norte da amostra;
* `LonDim` (int): graus de longitude leste da amostra;
* `value` (float): concentração de $CO_2$ em ppm;

# Tratamento

* Define uma mesma data de início para ambos os datasets
* Realiza correção de tipos de dados e padroniza colunas

In [10]:
# Padroniza datasets iniciando em 1850-01-01
tpr_data = tpr_data[tpr_data['dt'] >= '1850-01-01']

In [11]:
# Padroniza nomes de colunas entre datasets

trp_columns_rename = [
    ('dt', 't_date'),
    ('AverageTemperature', 't_temperature'),
    ('AverageTemperatureUncertainty', 't_temperature_unc'),
    ('City', 't_city'),
    ('Country', 't_country'),
    ('Latitude', 't_latitude'),
    ('Longitude', 't_longitude'),
]

co2_columns_rename = [
    ('Times', 'c_date'),
    ('LatDim', 'c_latitude'),
    ('LonDim', 'c_longitude'),
    ('value', 'c_co2'),
]

for old, new in trp_columns_rename:
    tpr_data = tpr_data.withColumnRenamed(old, new)

for old, new in co2_columns_rename:
    co2_data = co2_data.withColumnRenamed(old, new)

In [12]:
# Retira 'N' e 'E' do final dos valores de 
# latitude/longitude e converte para float

to_strip = ['t_latitude', 't_longitude']

for col in to_strip:
    tpr_data = tpr_data.withColumn(
        col,
        expr(f"float(substring({col}, 1, length({col})-1))")
    )

    tpr_data = tpr_data.withColumn(
        col + '_rnd',
        expr(f"int(round({col}))")
    )

In [13]:
tpr_data.withColumn(
    col,
    expr(f"float(substring({col}, 1, length({col})-1))")
)

DataFrame[t_date: string, t_temperature: string, t_temperature_unc: string, t_city: string, t_country: string, t_latitude: float, t_longitude: float, t_latitude_rnd: int, t_longitude_rnd: int]

In [14]:
tpr_data.show(5)

+----------+--------------------+-----------------+------+---------+----------+-----------+--------------+---------------+
|    t_date|       t_temperature|t_temperature_unc|t_city|t_country|t_latitude|t_longitude|t_latitude_rnd|t_longitude_rnd|
+----------+--------------------+-----------------+------+---------+----------+-----------+--------------+---------------+
|1850-01-01|              -5.265|             1.82| Århus|  Denmark|     57.05|      10.33|            57|             10|
|1850-02-01|               1.859|            1.641| Århus|  Denmark|     57.05|      10.33|            57|             10|
|1850-03-01|0.031999999999999806|            3.167| Århus|  Denmark|     57.05|      10.33|            57|             10|
|1850-04-01|  5.7639999999999985|            1.903| Århus|  Denmark|     57.05|      10.33|            57|             10|
|1850-05-01|              11.037|            0.586| Århus|  Denmark|     57.05|      10.33|            57|             10|
+----------+----

In [15]:
co2_data.show(5)

+----------+----------+-----------+-----------------+
|    c_date|c_latitude|c_longitude|            c_co2|
+----------+----------+-----------+-----------------+
|1850-01-01|         0|          0|288.1340637207031|
|1850-01-01|         0|          1|288.1340637207031|
|1850-01-01|         0|          2|288.1340637207031|
|1850-01-01|         0|          3|288.1340637207031|
|1850-01-01|         0|          4|288.1340637207031|
+----------+----------+-----------+-----------------+
only showing top 5 rows



***

Escrita dos datasets parseados, comentar se não desejado. Somente um por vez, haja vista que os arquivos são sobrescritos.

In [16]:
# tpr_data.coalesce(1).write.format('csv').option('header', True).mode('overwrite').save('./out/parsed')

In [17]:
# co2_data.coalesce(1).write.format('csv').option('header', True).mode('overwrite').save('./out/parsed')